In [1]:
import numpy as np
from pyrep import PyRep
from arm import CtRobot
from DH_dynamics import dh_robot_config
import sympy as sp
import os
import transforms3d as t3d
from pyrep.backend import vrep

pi = np.pi

In [2]:
pr = PyRep()
pr.launch(os.getcwd() + '/in-bore.ttt', headless=True)

In [3]:
pr.start()
ct_robot = CtRobot(name='inbore_arm', num_joints=4, joint_type=['r','r','r','p'])

## My calibration (based on vrep 'reference_frame')

In [4]:
# Grab modified DH parameters from V-REP using the dummy 'DH_frame'
# Need to set up their orientation following modified DH convention
D = []
theta = []
alpha = []
a = []
L = []

jointType=['r','r','r','p']
for i in range(ct_robot._num_joints):
    j = i+1
    tmp_alpha = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])
    tmp_a = ct_robot.DH_frames[i+1].get_position(relative_to=ct_robot.DH_frames[i])
    if tmp_alpha[0] < -1e-5 or tmp_alpha[0] > 1e-5:
        alpha.append(tmp_alpha[0])
    else:
        alpha.append(0)
    a.append(tmp_a[0])
    
    tmp_d = -ct_robot.DH_frames[j-1].get_position(relative_to=ct_robot.DH_frames[j])[2]
    tmp_theta = -ct_robot.DH_frames[j-1].get_orientation(relative_to=ct_robot.DH_frames[j])[2]
    if tmp_theta < -1e-5 or tmp_theta > 1e-5:
        theta.append(tmp_theta)
    else:
        theta.append(0)   
    D.append(tmp_d)
# Center of mass of link with respect to each joint    
for i in range(ct_robot._num_joints+1):
    L.append(ct_robot.COMs[i].get_position(relative_to=ct_robot.DH_frames[i]))
L = np.array(L)

Tbase = np.concatenate((np.array(ct_robot.DH_frames[0].get_matrix()).reshape(3,4), np.array([[0,0,0,1]])))

num_joints = ct_robot._num_joints

In [5]:
M = np.array([
        [0.1562, 0.1562, 0.1562, 3.324e-4, 1.484e-1, 1.484e-1],
        [3.126e-2, 3.126e-2, 3.126e-2, 1.470e-4, 4.065e-1, 4.064-1],
        [2.228e-2, 2.228e-2, 2.228e-2, 1.109e-4, 4.983e-1, 4.984e-1],
        [2.457e-2, 2.457e-2, 2.457e-2, 8.041e-4, 6.199e-1, 6.192e-1],
        [6.63e-3, 6.63e-3, 6.63e-3, 7.283e-5, 6.231e-1, 6.231e-1]
])

In [6]:
robot = dh_robot_config(num_joints, alpha, theta, D, a, jointType, Tbase, L, M)

In [7]:
robot.initKinematicTransforms()

Calculating link 0
Calculating link 1
Calculating link 2
Calculating link 3
Calculating link 4
Calculating Mass and Gravity Matrix...
-------------------------------------------------------------------
Calculation complete


## V-REP

In [8]:
for i in range(num_joints):
    print('---------------------------')
    print('Joint {}'.format(i+1))
    print('D: {}, theta: {}, a[i-1]: {}, alpha[i-1]: {}'.format(D[i],theta[i],a[i],alpha[i]))

---------------------------
Joint 1
D: 7.450580596923828e-08, theta: 1.570796251296997, a[i-1]: -5.960464477539063e-08, alpha[i-1]: 1.5708017349243164
---------------------------
Joint 2
D: 4.500150680541992e-06, theta: -1.5324003470595926e-05, a[i-1]: 0.06996411085128784, alpha[i-1]: 1.5707961320877075
---------------------------
Joint 3
D: -2.246815711259842e-08, theta: -1.570796251296997, a[i-1]: 0.06981068849563599, alpha[i-1]: 1.5707964897155762
---------------------------
Joint 4
D: 0.047888100147247314, theta: -2.19163757719798e-05, a[i-1]: 2.205371856689453e-06, alpha[i-1]: -1.570800542831421


## DH-FK

In [9]:
set_jot_pos = [0.1, 0.2, -0.2,0]
ct_robot.set_joint_positions(set_jot_pos)

In [10]:
joint_pos = [ct_robot.joints[i].get_joint_position() for i in range(ct_robot._num_joints)]
print('Joint position is', joint_pos)
print('End-joint position is', ct_robot._ik_tip.get_position())
print('End-joint orientation is', ct_robot._ik_tip.get_orientation())

Joint position is [0.10000000149011612, 0.20000000298023224, -0.20000000298023224, 0.0]
End-joint position is [0.022745875641703606, -0.7929102182388306, 0.3352990448474884]
End-joint orientation is [-2.8454768657684326, -0.039479054510593414, -0.19611364603042603]


In [11]:
err_j2j = 0

for i in range(num_joints):
    #vrep
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix(ct_robot.DH_frames[i])).reshape(3,4)
    
    # FK
    Tp = sp.lambdify(robot.q, robot._Tj2j[i])(*tuple(joint_pos))
    print('Joint %d: diff:'%(i+1), np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-3:
        err_j2j += 1
        print('Wrong match of joint %d'%(i+1))
        print('getting from V-REP')
        print(T)
        print('----------------------------------')
        print('getting from FK')
        print(Tp)
        print('----------------------------------')


Joint 1: diff: 2.154409313600187e-05
Joint 2: diff: 7.863339088194807e-05
Joint 3: diff: 0.00038716847900554765
Joint 4: diff: 5.137486143653346e-05


In [12]:
err_j2b = 0
for i in range(num_joints):
    #vrep
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix()).reshape(3,4)
    
    # FK
    Tp = sp.lambdify(robot.q, robot._Tjoint[i])(*tuple(joint_pos))
    print('Joint %d: diff:'%(i+1), np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-3:
        err_j2b += 1
        print('Wrong match of joint %d'%(i+1))
        print('getting from V-REP')
        print(T)
        print('----------------------------------')
        print('getting from FK')
        print(Tp)
        print('----------------------------------')

Joint 1: diff: 2.1683712552700328e-05
Joint 2: diff: 7.514882554897797e-05
Joint 3: diff: 0.0004611840904792975
Joint 4: diff: 0.00046373477327257357


## Save

In [13]:
param = ['D', 'a', 'alpha', 'theta', 'num_joints', 'jointType', 'Tbase', 'L', 'M']
if err_j2b == 0 and err_j2j == 0:
    print('Check FK library with V-REP correct')
    print('--------------------------------------------------')
    print('Writing DH parameters, Mass and COM information into file')
    for i in range(len(param)):
        np.save('./robot_config/inbore_config/%s'%param[i], eval(param[i]))
    print('--------------------------------------------------')
    print('Writing complete')
else:
    print('Please check DH convention with V-REP')

Check FK library with V-REP correct
--------------------------------------------------
Writing DH parameters, Mass and COM information into file
--------------------------------------------------
Writing complete


In [14]:
pr.stop()
pr.shutdown()